In [ ]:
from scipy.integrate import odeint 
import time
import math
import numpy as np
import pylab as py
from math import *
import numpy as np
from scipy.optimize import newton
from matplotlib import animation, rc
from matplotlib import pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from IPython.display import HTML

import warnings
import os

In [ ]:
# Differential equations describing the system
def double_pendulum(u,t,m1,m2,L1,L2,g):
    # du = derivatives
    # u = variables
    # p = parameters
    # t = time variable

    du = np.zeros(4)
    c = np.cos(u[0]-u[2])  # intermediate variables
    s = np.sin(u[0]-u[2])  # intermediate variables

    du[0] = u[1]   # d(theta 1)
    du[1] = ( m2*g*np.sin(u[2])*c - m2*s*(L1*c*u[1]**2 + L2*u[3]**2) - (m1+m2)*g*np.sin(u[0]) ) /( L1 *(m1+m2*s**2) )
    du[2] = u[3]   # d(theta 2)
    du[3] = ((m1+m2)*(L1*u[1]**2*s - g*np.sin(u[2]) + g*np.sin(u[0])*c) + m2*L2*u[3]**2*s*c) / (L2 * (m1 + m2*s**2))

    return du

In [ ]:
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

def preprocess_sol(train_data_normalized, train_window):
    train_inout_seq= create_inout_sequences(train_data_normalized.reshape(len(train_data_normalized), 9), train_window)
    return train_inout_seq

In [ ]:
def makepath_param(theta1, theta2, m1,m2,l1,l2, g = 9.81):
    u0 = [np.pi*theta1/180, 0, np.pi*theta2/180, 0]
    tfinal = 125.0 
    Nt = 3755
    #timesteps
    dt = tfinal / Nt
    
    t = np.linspace(0, tfinal, Nt)
    sol = odeint(double_pendulum, u0, t, args=(m1,m2,l1,l2,g))
    
    # add parameter in columns
    params = np.array([m1, m2, l1, l2, dt])
    
    # repeat params along the length of the simulation
    params_repeated = np.tile(params, (sol.shape[0], 1))
    
    # stack it on the simulation
    sol_with_params = np.hstack((sol, params_repeated))
    
    return sol_with_params

In [ ]:
def multi_simulations(theta1, theta2, change_of_angle, amount_of_different_inits, m1_params, m2_params, l1_params, l2_params, g):
    simulation_sets = []
    #This slope is defined for different initial condition
    for i in range(0, amount_of_different_inits):
        theta1 += change_of_angle
        theta2 -= change_of_angle
        print(f"theta 1 ist {theta1}")
        print(f"theta 2 ist {theta2}")
        for m1 in m1_params:
            for m2 in m2_params:
                for l1 in l1_params:
                    for l2 in l2_params:
                        sol = makepath_param(theta1, theta2, m1,m2,l1,l2,g)
                        print(f"m1 = {m1}; l1 = {l1}; m2 = {m2}; l2 = {l2}\n")
                        simulation_sets.append(sol)
    return simulation_sets

In [ ]:
def data_splitter_and_scaler(scaler, Simulation_datasets, test_size = 0.2):
    train_splits = []
    test_splits = []

    for dataset in Simulation_datasets:
        split_index = int(len(dataset) * (1 - test_size))
        train_split = dataset[:split_index]
        test_split = dataset[split_index:]
        train_splits.append(train_split)
        test_splits.append(test_split)
    
    for i in range(len(train_splits)):
        dataset = train_splits[i]
        train_window = 10
        
        #separating the params since those valuus shouldnt be scale
        params_and_dt = dataset[:, 4:]
        m1, m2, l1, l2, dt = params_and_dt[0]
        print(f"m1 = {m1}; l1 = {l1}; m2 = {m2}; l2 = {l2}; dt = {dt}\n")
        
        train_data_normalized = scaler.fit_transform(dataset[:, :4].reshape(-1, 4))
        train_data_normalized_with_params = np.concatenate((train_data_normalized, params_and_dt), axis=1)    
        train_data_normalized_with_params = torch.FloatTensor(train_data_normalized_with_params).reshape(len(dataset), 9)

        #A change of the reshape function inside the preprocess_sol might be required in case you change the amount of timesteps
        train_inout_seq = preprocess_sol(train_data_normalized_with_params, train_window)
        
        train_splits[i] = train_inout_seq
        
    return train_splits, test_splits

In [ ]:
def energy_calculation(theta1, theta2, theta1_dot, theta2_dot, mass1=2.0, mass2=1.0, length1=1.4, length2=1.0, g=9.81):

    kinetic_energy = 0.5 * mass1 * length1**2 * theta1_dot**2 + 0.5 * mass2 * ((length1 * theta1_dot)**2 + (length2 * theta2_dot)**2 + 2 * length1 * length2 * theta1_dot * theta2_dot * np.cos(theta1 - theta2))

    potential_energy = -mass1 * g * length1 * np.cos(theta1) - mass2 * g * (length1 * np.cos(theta1) + length2 * np.cos(theta2))

    total_energy = abs(kinetic_energy + potential_energy)

    return total_energy

In [ ]:
#this is the full OneStep function.   it
def OneStep(model, model_layers, data, filename, filepath_excel, steps = 100, g = 9.81):
    print('data set length =', len(data))
    train_window = 10
    initial_predict_data = data[:, :4]
    scaler = MinMaxScaler(feature_range=(-1, 1))
    train_data_normalized = scaler.fit_transform(initial_predict_data.reshape(-1, 4))
    fut_pred = len(data) -train_window
    test_inputs = train_data_normalized[0:train_window].reshape(train_window,4).tolist()
    #print(test_inputs)
    # s2 = train_data_normalized.reshape(len(data),4).tolist()
    realdata = data
    model.eval().cpu()
    preds = test_inputs.copy()
    t2 = test_inputs
    hidden_layer_size = 100
    x = 0
    for i in range(fut_pred):
        seq = torch.FloatTensor(t2[i:]).cpu()
        model.c_h = (torch.zeros(model_layers, 1, hidden_layer_size),
                        torch.zeros(model_layers, 1, hidden_layer_size))
        x = model(seq).cpu()
        x = x.cpu()
        preds.append(x.detach().numpy())
        t2.append(x.detach().numpy())
    actual_predictions = scaler.inverse_transform(np.array(preds ).reshape(-1,4))
    print(len(actual_predictions))

    # the following will plot the lower mass path for steps using the actual ODE sover
    # and the predicitons
    plt.figure( figsize=(10,5))
    u0 = data[:,0]     # theta_1
    u1 = data[:,1]     # omega 1
    u2 = data[:,2]     # theta_2
    u3 = data[:,3]     # omega_2
    up0 = actual_predictions[:,0]     # theta_1
    up1 = actual_predictions[:,1]     # omega 1
    up2 = actual_predictions[:,2]     # theta_2
    up3 = actual_predictions[:,3]     # omega_2
    m1 = data[:, 4][0]
    m2 = data[:, 5][0]
    l1 = data[:, 6][0]
    l2 = data[:, 7][0]
    x1 = l1*np.sin(u0);          # First Pendulum
    y1 = -l1*np.cos(u0);
    x2 = x1 + l2*np.sin(u2);     # Second Pendulum
    y2 = y1 - l2*np.cos(u2);
    xp1 = l1*np.sin(up0);          # First Pendulum
    yp1 = -l1*np.cos(up0);
    xp2 = xp1 + l2*np.sin(up2);     # Second Pendulum
    yp2 = yp1 - l2*np.cos(up2); 
    print(x2[0], y2[0])
    plt.plot(x2[0:steps], y2[0:steps], color='r')
    plt.plot(xp2[0:steps],yp2[0:steps] , color='g')

    ###############################################################################
    dt = data[:, 8][0]
    Nt = len(data[:, 8])
    tfinal = dt * Nt
    t = np.linspace(0, tfinal, Nt)

    energy_data = energy_calculation(u0, u2, u1, u3, m1, m2, l1, l2)
    energy_predicted = energy_calculation(up0, up2, up1, up3, m1, m2, l1, l2)

    ###############################################################################
    plt.figure(figsize=(10, 6))
    ## Only necessary when addapting the lookback
    # lookback += 200
    plt.suptitle('Simulation vs Prediction - ' + filename, fontsize=16)

    lookback = 0
    plt.subplot(3, 2, 1)
    plt.plot(t[lookback:], data[lookback:, 0], label='Simulation')
    plt.plot(t[lookback:], actual_predictions[:, 0], label='Prediction')
    plt.ylabel('Theta 1')
    plt.legend()

    plt.subplot(3, 2, 2)
    plt.plot(t[lookback:], data[lookback:, 1], label='Simulation')
    plt.plot(t[lookback:], actual_predictions[:, 1], label='Prediction')
    plt.ylabel('Omega 1')
    plt.legend()

    plt.subplot(3, 2, 3)
    plt.plot(t[lookback:], data[lookback:, 2], label='Simulation')
    plt.plot(t[lookback:], actual_predictions[:, 2], label='Prediction')
    plt.ylabel('Theta 2')
    plt.xlabel('Time in s')
    plt.legend()

    plt.subplot(3, 2, 4)
    plt.plot(t[lookback:], data[lookback:, 3], label='Simulation')
    plt.plot(t[lookback:], actual_predictions[:, 3], label='Prediction')
    plt.ylabel('Omega 2')
    plt.xlabel('Time in s')
    plt.legend()

    plt.subplot(3, 2, 5)
    plt.plot(t[lookback:], energy_data, label='Simulation')
    plt.plot(t[lookback:], energy_predicted, label='Prediction')
    plt.ylabel('Omega 2')
    plt.xlabel('Time in s')
    plt.legend()

    ##############################################################################
    
    # Create dataframe
    df = pd.DataFrame({ 'time': t,
        f'theta1_actual': data[:, 0], 'omega1_actual': data[:, 1],
        'theta2_actual': data[:, 2], 'omega2_actual': data[:, 3],
        'energy_actual': energy_data,
        'm1': data[:, 4], 'm2': data[:, 5], 'l1': data[:, 6], 'l2': data[:, 7], 'dt': data[:, 8],
        'theta1_predicted': actual_predictions[:, 0], 'omega1_predicted': actual_predictions[:, 1],
        'theta2_predicted': actual_predictions[:, 2], 'omega2_predicted': actual_predictions[:, 3],
        'energy_predicted': energy_predicted,
    })

    #Combination filepath and filename
    full_path = os.path.join(filepath_excel, filename)

    # safe csv
    df.to_csv(f'{full_path}.csv', index=False)
    
    return actual_predictions

In [ ]:
def get_positions(theta1, theta2, length1, length2):
    x1 = length1 * np.sin(theta1)
    y1 = -length1 * np.cos(theta1)
    x2 = x1 + length2 * np.sin(theta2)
    y2 = y1 - length2 * np.cos(theta2)
    return x1, y1, x2, y2